In [ ]:
# === Celda 1: Imports, rutas, device y parámetros ===
from pathlib import Path
import time, random
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# ---- Rutas (ajústalas si hace falta) ----
DATA_DIR = Path(r"D:/proyectos/Caso_aprendizaje-Melanomas/data/raw")
TRAIN_DIR = DATA_DIR / "train"
VAL_DIR   = DATA_DIR / "val"
TEST_DIR  = DATA_DIR / "test"

for p in (TRAIN_DIR, VAL_DIR, TEST_DIR):
    print(p, "exists:", p.exists())

# ---- Semillas fijas ----
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# ---- Device (GPU si hay) ----
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# ---- Parámetros de datos/entreno ----
CLASSES = 2
ROWS = COLS = 224
BATCH = 16
EPOCHS = 15
LR = 1e-3
NUM_WORKERS = 0            # En Windows, 0 es lo más estable. Si todo ok, prueba 2.
USE_AMP = True             # Mixed precision en GPU; en CPU se ignora.

# Normalización (ImageNet)
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD  = (0.229, 0.224, 0.225)


In [ ]:
# === Celda 2: Transforms (tratado de imagen) + DataLoaders ===
train_tfms = transforms.Compose([
    transforms.Resize((ROWS, COLS)),

    # --- Augmentaciones ligeras ---
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15, fill=tuple(int(m*255) for m in IMAGENET_MEAN)),  # evita bordes negros

    # --- Luz/contraste ---
    transforms.ColorJitter(brightness=0.15, contrast=0.15),   # ±15%
    transforms.RandomAutocontrast(p=0.3),

    # --- Bordes/nitidez (suave) ---
    transforms.RandomAdjustSharpness(sharpness_factor=1.5, p=0.3),
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 0.8)),  # opcional, suaviza ruido fino

    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

val_tfms = transforms.Compose([
    transforms.Resize((ROWS, COLS)),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

# Datasets
train_ds = datasets.ImageFolder(TRAIN_DIR, transform=train_tfms)
val_ds   = datasets.ImageFolder(VAL_DIR,   transform=val_tfms)
print("Clases:", train_ds.classes)
assert len(train_ds.classes) == CLASSES, f"Esperaba {CLASSES} clases."

# DataLoaders
train_dl = DataLoader(train_ds, batch_size=BATCH, shuffle=True,
                      num_workers=NUM_WORKERS, pin_memory=True)
val_dl   = DataLoader(val_ds,   batch_size=BATCH, shuffle=False,
                      num_workers=NUM_WORKERS, pin_memory=True)


In [ ]:
# === Celda 3: Definición del modelo CNN ===
class SmallCNN(nn.Module):
    def __init__(self, in_ch=3, num_classes=2, drop_conv=0.20, drop_fc=0.5):
        super().__init__()
        self.block1 = nn.Sequential(
            nn.Conv2d(in_ch, 32, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(32), nn.ReLU(inplace=True),
            nn.MaxPool2d(2,2), nn.Dropout(drop_conv),
        )
        self.block2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64), nn.ReLU(inplace=True),
            nn.MaxPool2d(2,2), nn.Dropout(drop_conv),
        )
        self.block3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(128), nn.ReLU(inplace=True),
            nn.MaxPool2d(2,2), nn.Dropout(drop_conv),
        )
        self.block4 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(256), nn.ReLU(inplace=True),
            nn.MaxPool2d(2,2), nn.Dropout(drop_conv),
        )
        self.gap = nn.AdaptiveAvgPool2d((1,1))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(drop_fc),
            nn.Linear(256, num_classes),
        )

    def forward(self, x):
        x = self.block1(x)   # 224 -> 112
        x = self.block2(x)   # 112 -> 56
        x = self.block3(x)   # 56  -> 28
        x = self.block4(x)   # 28  -> 14
        x = self.gap(x)      # -> [B,256,1,1]
        x = self.classifier(x)  # -> [B,2]
        return x

model = SmallCNN(in_ch=3, num_classes=CLASSES).to(device)
params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Parámetros entrenables: {params:,}")


In [ ]:
# === Celda 4: Loss, optimizador y métrica ===
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

def accuracy_from_logits(logits, targets):
    preds = logits.argmax(dim=1)
    return (preds == targets).float().mean().item()


In [ ]:
# === Celda 5: Entreno/validación ===
scaler = torch.amp.GradScaler(enabled=USE_AMP and device.type == "cuda")

def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss, total_acc, n = 0.0, 0.0, 0
    for x, y in loader:
        x = x.to(device, non_blocking=True)
        y = y.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)
        with torch.amp.autocast(device_type="cuda", dtype=torch.float16,
                                enabled=USE_AMP and device.type == "cuda"):
            logits = model(x)
            loss = criterion(logits, y)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()
        total_acc  += accuracy_from_logits(logits, y)
        n += 1
    return total_loss/n, total_acc/n

@torch.no_grad()
def validate(model, loader, criterion):
    model.eval()
    total_loss, total_acc, n = 0.0, 0.0, 0
    for x, y in loader:
        x = x.to(device, non_blocking=True)
        y = y.to(device, non_blocking=True)
        logits = model(x)
        loss = criterion(logits, y)
        total_loss += loss.item()
        total_acc  += accuracy_from_logits(logits, y)
        n += 1
    return total_loss/n, total_acc/n

history = {"train_loss":[], "train_acc":[], "val_loss":[], "val_acc":[]}
BEST_PATH = "smallcnn_simple_best.pth"
best_val = float("inf")

for epoch in range(1, EPOCHS+1):
    t0 = time.time()
    tr_loss, tr_acc = train_one_epoch(model, train_dl, optimizer, criterion)
    va_loss, va_acc = validate(model, val_dl, criterion)

    history["train_loss"].append(tr_loss); history["train_acc"].append(tr_acc)
    history["val_loss"].append(va_loss);   history["val_acc"].append(va_acc)

    if va_loss < best_val:
        best_val = va_loss
        torch.save({
            "model": model.state_dict(),
            "classes": train_ds.classes,
            "img_size": (ROWS, COLS)
        }, BEST_PATH)

    dt = time.time() - t0
    print(f"[{epoch:02d}/{EPOCHS}] "
          f"train_loss={tr_loss:.4f} acc={tr_acc:.4f} | "
          f"val_loss={va_loss:.4f} acc={va_acc:.4f} | {dt:.1f}s")

print("Mejor modelo guardado en:", BEST_PATH)
